### Claculate probabilistic coreness

May 29, 2025

Lorena Lopez Dominguez

kernel: base (Python 3.12.8)

In [1]:
import os
import re
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.spatial.distance import squareform

In [3]:
# --- Define core-periphery model fitting function ---
def fit_coreness(A, max_iter=1000):
    """Estimate probabilistic coreness from adjacency matrix A (undirected, symmetric)"""
    n = A.shape[0]
    x0 = np.random.rand(n)
    x0 /= np.linalg.norm(x0)  # normalize initial guess

    def loss(x):
        x = np.clip(x, 0, 1)
        X = np.outer(x, x)
        return np.sum((A - X) ** 2)

    result = minimize(loss, x0, bounds=[(0, 1)] * n, options={'maxiter': max_iter})
    return np.clip(result.x, 0, 1)

In [4]:
# --- Paths ---
base_dir = '/Users/Lore/Desktop/brainhack/project/BrainHack-Project-'
data_dir = os.path.join(base_dir, 'Data')
results_dir = os.path.join(base_dir, 'results')
os.makedirs(results_dir, exist_ok=True)

In [5]:
# --- Store all subjects’ coreness scores ---
all_coreness = pd.DataFrame()

In [6]:
# --- Loop over subjects ---
for filename in os.listdir(data_dir):
    if filename.startswith('data_sub-') and filename.endswith('.txt'):
        subject_id = re.search(r'data_sub-(.*)\.txt', filename).group(1)
        file_path = os.path.join(data_dir, filename)

        # Load FC matrix (skip header)
        A = np.loadtxt(file_path, delimiter=',', skiprows=1)

        # Ensure matrix is symmetric
        A = (A + A.T) / 2

        # Optional: zero diagonal
        np.fill_diagonal(A, 0)

        # Fit coreness
        coreness_scores = fit_coreness(A)

        # Store with subject ID
        all_coreness[subject_id] = coreness_scores

In [7]:
all_coreness

,A00038411,A00075896,A00057406,A00029127,A00056452,A00062292,A00082216,A00039669,A00074769,A00062282,...,A00055447,A00033834,A00055806,A00032876,A00058503,A00033832,A00035926,A00054857,A00065487,A00085406
0,0.726407,0.558761,0.394837,0.678373,0.616415,0.837606,0.681109,0.651749,0.719172,0.636403,...,0.750170,0.717886,0.712342,0.531635,0.805571,0.716686,0.644191,0.656565,0.682479,0.661613
1,0.629276,0.630741,0.586669,0.796931,0.864651,0.814957,0.489135,0.788320,0.736708,0.760355,...,0.674621,0.664211,0.765990,0.624089,0.793506,0.739941,0.581868,0.800556,0.628445,0.640053
2,0.551123,0.427128,0.721467,0.686211,0.604578,0.698402,0.629167,0.691592,0.450678,0.195361,...,0.842356,0.586870,0.334651,0.250976,0.757933,0.624713,0.562457,0.501133,0.718998,0.739204
3,0.597180,0.356213,0.531091,0.559457,0.835158,0.813221,0.408932,0.742437,0.676535,0.259896,...,0.782735,0.552127,0.527894,0.380442,0.808191,0.729661,0.722886,0.751667,0.692061,0.849497
4,0.319509,0.256200,0.632793,0.509819,0.830995,0.291120,0.401691,0.620503,0.560392,0.297522,...,0.838039,0.333657,0.461210,0.407848,0.739076,0.388295,0.568111,0.685088,0.395399,0.738034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,0.606466,0.739840,0.540646,0.580471,0.578511,0.652463,0.785655,0.485104,0.782657,0.524292,...,0.595802,0.520603,0.679828,0.599821,0.389392,0.616328,0.696142,0.294420,0.538340,0.537854
416,0.599655,0.685371,0.173212,0.735224,0.606264,0.687660,0.680205,0.366045,0.841684,0.759970,...,0.704893,0.512044,0.351694,0.674140,0.516229,0.777922,0.622403,0.360233,0.561416,0.744395
417,0.389745,0.736528,0.642240,0.713623,0.840976,0.775358,0.690227,0.484962,0.651393,0.562927,...,0.654374,0.746758,0.560334,0.600414,0.645350,0.651640,0.339427,0.724527,0.598381,0.528526
418,0.516231,0.813682,0.707900,0.699123,0.808508,0.683172,0.871589,0.357291,0.739066,0.858977,...,0.748455,0.660729,0.602050,0.464576,0.730477,0.614558,0.563872,0.499155,0.607869,0.664737


In [ ]:
# --- Format ---
coreness_df = all_coreness.T.reset_index()
coreness_df.rename(columns={'index': 'ID'}, inplace=True)
coreness_df.columns = ['ID'] + [f'ROI_{i+1}' for i in range(coreness_df.shape[1] - 1)]

In [9]:
coreness_df

,ID,ROI_1,ROI_2,ROI_3,ROI_4,ROI_5,ROI_6,ROI_7,ROI_8,ROI_9,...,ROI_411,ROI_412,ROI_413,ROI_414,ROI_415,ROI_416,ROI_417,ROI_418,ROI_419,ROI_420
0,A00038411,0.726407,0.629276,0.551123,0.597180,0.319509,0.451824,0.407030,0.558016,0.453666,...,0.380350,0.683771,0.773778,0.484414,0.447763,0.606466,0.599655,0.389745,0.516231,0.617358
1,A00075896,0.558761,0.630741,0.427128,0.356213,0.256200,0.344303,0.171080,0.709632,0.480697,...,0.745989,0.694554,0.474472,0.703586,0.642191,0.739840,0.685371,0.736528,0.813682,0.753335
2,A00057406,0.394837,0.586669,0.721467,0.531091,0.632793,0.327892,0.599174,0.460955,0.285844,...,0.730098,0.719658,0.549815,0.533289,0.487333,0.540646,0.173212,0.642240,0.707900,0.556608
3,A00029127,0.678373,0.796931,0.686211,0.559457,0.509819,0.558387,0.242426,0.814439,0.580657,...,0.554339,0.798189,0.724333,0.448844,0.397539,0.580471,0.735224,0.713623,0.699123,0.554149
4,A00056452,0.616415,0.864651,0.604578,0.835158,0.830995,0.845225,0.807642,0.857806,0.849957,...,0.718089,0.832015,0.566451,0.665888,0.558193,0.578511,0.606264,0.840976,0.808508,0.656703
5,A00062292,0.837606,0.814957,0.698402,0.813221,0.291120,0.339357,0.643938,0.613840,0.557884,...,0.754828,0.880346,0.900527,0.683940,0.549210,0.652463,0.687660,0.775358,0.683172,0.753168
6,A00082216,0.681109,0.489135,0.629167,0.408932,0.401691,0.405156,0.379822,0.747217,0.291899,...,0.751110,0.835306,0.899404,0.588239,0.790484,0.785655,0.680205,0.690227,0.871589,0.634354
7,A00039669,0.651749,0.788320,0.691592,0.742437,0.620503,0.755248,0.673005,0.808508,0.743720,...,0.651256,0.693884,0.667073,0.427679,0.470057,0.485104,0.366045,0.484962,0.357291,0.234582
8,A00074769,0.719172,0.736708,0.450678,0.676535,0.560392,0.581903,0.690218,0.787770,0.535360,...,0.560333,0.875215,0.863432,0.694748,0.842041,0.782657,0.841684,0.651393,0.739066,0.737455
9,A00062282,0.636403,0.760355,0.195361,0.259896,0.297522,0.169399,0.044654,0.714279,0.560373,...,0.450682,0.763123,0.832675,0.614366,0.373614,0.524292,0.759970,0.562927,0.858977,0.634815


In [10]:
# --- Save ---
coreness_df.to_csv(os.path.join(results_dir, 'probabilistic_coreness_all.csv'), index=False)

#print("Saved probabilistic coreness scores to:")
#print(os.path.join(results_dir, 'probabilistic_coreness_all.csv'))
